Install libraries etc

In [ ]:
# https://www.geeksforgeeks.org/text-preprocessing-for-nlp-tasks/

In [13]:
!pip install numpy pandas
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [14]:
import re
import string
import pandas as pd
import nltk
# import contractions
# from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

yelp_df = pd.read_csv("yelp.csv")

# CLEAN TEXT

def clean(text):
  text = text.lower()
  text = re.sub(r'\d+', '', text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub(r'\W', ' ', text)
  return text

yelp_df['text'] = yelp_df['text'].apply(clean)

# tokenization
yelp_df['text'] = yelp_df['text'].apply(word_tokenize)

# stopwords

stop_words = set(stopwords.words('english'))

def remove_stop(text):
  return [word for word in text if word not in stop_words]

yelp_df['text'] = yelp_df['text'].apply(remove_stop)

# lemmatization

lemmatizer = WordNetLemmatizer()

def lem(text):
  return [lemmatizer.lemmatize(word) for word in text]

yelp_df['text'] = yelp_df['text'].apply(lem)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
# change stars so that there are 3 categories

def change_stars(stars):
  if stars > 3:
    return 2
  elif stars < 3:
    return 0
  else:
    return 1


yelp_df["stars"] = yelp_df["stars"].apply(change_stars)



In [16]:
# word embedding using glove

import numpy as np


# load
def load_glove(path):
  embeddings = {}
  with open(path, 'r', encoding = 'utf-8') as f:
    for line in f:
      word_vector = line.strip().split()
      word = word_vector[0]
      vector = np.array(word_vector[1:], dtype=np.float32)
      embeddings[word] = vector
  return embeddings

glove_path = "glove.6B.50d.txt"
glove = load_glove(glove_path)

# converting the vectors

def text_to_vector(text, glove, dim=50):
  vector = []
  for word in text:
    if word in glove:
      vector.append(glove[word])
    else:
      vector.append(np.zeros(dim))
  return vector

yelp_df['vector'] = yelp_df['text'].apply(lambda text: text_to_vector(text, glove))



              business_id        date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg  2011-01-26  fWKvX83p0-ka4JS3dc6E5A      2   
1  ZRJwVLyzEJq1VAihDhYiow  2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      2   
2  6oRAC4uyJCsJl1X0WZpVSA  2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      2   
3  _1QQZuf4zZOyFCvXc0o6Vg  2010-05-27  G-WvGaISbqqaMHlNnByodA      2   
4  6ozycU1RpktNG2-1BroVtw  2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      2   

                                                text    type  \
0  [wife, took, birthday, breakfast, excellent, w...  review   
1  [idea, people, give, bad, review, place, go, s...  review   
2  [love, gyro, plate, rice, good, also, dig, can...  review   
3  [rosie, dakota, love, chaparral, dog, park, co...  review   
4  [general, manager, scott, petello, good, egg, ...  review   

                  user_id  cool  useful  funny  \
0  rLtl8ZkDX5vH5nAx9C3q5Q     2       5      0   
1  0a2KyEL0d3Yb1V6aivbIuQ     0       0      0   
2  0hT2KtfLiobPvh6cDC8JQg     0 

In [17]:
# padding

max_length = 300

def padding(seq):
  if len(seq) >= max_length:
        return np.array(seq[:max_length])
  elif len(seq) < max_length:
    padded = [np.zeros(50) for _ in range(max_length - len(seq))]
    return np.array(seq + padded)
  else:
    print("else")
    return [np.zeros(50) for _ in range(max_length)]

yelp_df["pad_vectors"] = yelp_df["vector"].apply(padding)


In [18]:
#sampling
from sklearn.model_selection import train_test_split

X = np.stack(yelp_df['pad_vectors'].values)
y = yelp_df['stars'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 47, stratify = y)


In [19]:
#LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking, Dropout, Bidirectional
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(max_length, 50)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, class_weight=class_weights, epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 185s 708ms/step - accuracy: 0.4067 - loss: 1.0745 - val_accuracy: 0.5755 - val_loss: 0.9278
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 174s 695ms/step - accuracy: 0.5701 - loss: 0.9342 - val_accuracy: 0.5555 - val_loss: 0.9095
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 173s 693ms/step - accuracy: 0.6015 - loss: 0.8983 - val_accuracy: 0.6210 - val_loss: 0.8679
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 210s 726ms/step - accuracy: 0.6461 - loss: 0.8172 - val_accuracy: 0.6615 - val_loss: 0.7895
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 200s 720ms/step - accuracy: 0.6605 - loss: 0.7796 - val_accuracy: 0.5525 - val_loss: 0.9355
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 179s 718ms/step - accuracy: 0.6794 - loss: 0.7434 - val_accuracy: 0.6170 - val_loss: 0.8492
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 178s 712ms/step - accuracy: 0.7154 - loss: 0.6837 - val_accuracy: 0.6780 - val_loss: 0.7519
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 198s 696ms/step - accuracy: 0.7166 -

In [20]:
from sklearn.metrics import classification_report

y_pred_probs = model.predict(X_test, batch_size=32, verbose=1)

y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 196ms/step
              precision    recall  f1-score   support

           0       0.53      0.59      0.55       335
           1       0.26      0.41      0.31       292
           2       0.87      0.73      0.80      1373

    accuracy                           0.66      2000
   macro avg       0.55      0.58      0.56      2000
weighted avg       0.72      0.66      0.69      2000



In [25]:
# import already cleaned dataset
nyt_test = pd.read_pickle("nyt_df.pkl")

X_nyt = np.stack(nyt_test['pad_vectors'].values)
y_nyt = nyt_test['stars'].values

In [27]:
#run model
y_pred_probs_nyt = model.predict(X_nyt, batch_size=32, verbose=1)
y_pred_nyt = np.argmax(y_pred_probs_nyt, axis=1)

print(classification_report(y_nyt, y_pred_nyt))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
              precision    recall  f1-score   support

           0       0.50      0.70      0.58        10
           1       0.67      0.20      0.31        10
           2       0.31      0.40      0.35        10

    accuracy                           0.43        30
   macro avg       0.49      0.43      0.41        30
weighted avg       0.49      0.43      0.41        30

